Title: 

Introduction: 

In [1]:
library(tidyverse)
library(tidymodels)
library(readxl)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

Preliminary exploratory data analysis:

1. Demonstrate that the dataset can be read from the web into R 

In [2]:
raisin_data <- read_csv("https://github.com/dylankim990/DSCI100_Group_2/raw/main/data/Raisin_Dataset.csv")

Parsed with column specification:
cols(
  Area = col_double(),
  MajorAxisLength = col_double(),
  MinorAxisLength = col_double(),
  Eccentricity = col_double(),
  ConvexArea = col_double(),
  Extent = col_double(),
  Perimeter = col_double(),
  Class = col_character()
)



2. Clean and wrangle your data into a tidy format

In [3]:
# tidying was not neccesary since they were in a tidy format
glimpse(raisin_data)
head(raisin_data)

Rows: 900
Columns: 8
$ Area            <dbl> 87524, 75166, 90856, 45928, 79408, 49242, 42492, 6095…
$ MajorAxisLength <dbl> 442.2460, 406.6907, 442.2670, 286.5406, 352.1908, 318…
$ MinorAxisLength <dbl> 253.2912, 243.0324, 266.3283, 208.7600, 290.8275, 200…
$ Eccentricity    <dbl> 0.8197384, 0.8018052, 0.7983536, 0.6849892, 0.5640113…
$ ConvexArea      <dbl> 90546, 78789, 93717, 47336, 81463, 51368, 43904, 6232…
$ Extent          <dbl> 0.7586506, 0.6841296, 0.6376128, 0.6995994, 0.7927719…
$ Perimeter       <dbl> 1184.040, 1121.786, 1208.575, 844.162, 1073.251, 881.…
$ Class           <chr> "Kecimen", "Kecimen", "Kecimen", "Kecimen", "Kecimen"…


Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
87524,442.2460,253.2912,0.8197384,90546,0.7586506,1184.040,Kecimen
75166,406.6907,243.0324,0.8018052,78789,0.6841296,1121.786,Kecimen
90856,442.2670,266.3283,0.7983536,93717,0.6376128,1208.575,Kecimen
45928,286.5406,208.7600,0.6849892,47336,0.6995994,844.162,Kecimen
79408,352.1908,290.8275,0.5640113,81463,0.7927719,1073.251,Kecimen
49242,318.1254,200.1221,0.7773513,51368,0.6584564,881.836,Kecimen


3. Summarizing the data in at least one table

In [4]:
#a) the number of observations in each class
obs_num_class <- raisin_data %>%
    group_by(Class) %>%
    summarize(num_obs = n())

obs_num_class

`summarise()` ungrouping output (override with `.groups` argument)



Class,num_obs
<chr>,<int>
Besni,450
Kecimen,450


In [5]:
#b) the means of the predictor variables you plan to use in your analysis
means_predictors <- raisin_data %>%
    select(-Class) %>%
    map_df(mean)

means_predictors    

Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
87804.13,430.93,254.4881,0.7815422,91186.09,0.6995079,1165.907


In [6]:
#c) counting how many rows have missing data. 
num_of_na <- colSums(is.na(raisin_data))

num_of_na

Area MajorAxisLength MinorAxisLength    Eccentricity      ConvexArea 
              0               0               0               0               0 
         Extent       Perimeter           Class 
              0               0               0

4. Visualizing the data with at least one plot relevant to the analysis

In [7]:
#a) comparing the distributions of each of the predictor variables

Methods:
1. Process of data analysis

The process of Raisin data analysis uses classification to predict if the test data are either Kecimen or Besni raisin. 

2. Variables/columns to use 

For each variable think: is this a useful variable for prediction?

In [13]:
raisin_split <- initial_split(raisin_data, prop = 0.75 ,strata = Class)
raisin_training <- training(raisin_split)
raisin_testing <- testing(raisin_split)

raisin_subset <- raisin_training %>%
  select(Area,
         MajorAxisLength, 
         MinorAxisLength, 
         Eccentricity, 
         ConvexArea, 
         Extent, 
         Perimeter, 
         Class)

names <- colnames(select(raisin_subset, -Class))

raisin_subset


Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
87524,442.2460,253.2912,0.8197384,90546,0.7586506,1184.040,Kecimen
75166,406.6907,243.0324,0.8018052,78789,0.6841296,1121.786,Kecimen
90856,442.2670,266.3283,0.7983536,93717,0.6376128,1208.575,Kecimen
45928,286.5406,208.7600,0.6849892,47336,0.6995994,844.162,Kecimen
79408,352.1908,290.8275,0.5640113,81463,0.7927719,1073.251,Kecimen
49242,318.1254,200.1221,0.7773513,51368,0.6584564,881.836,Kecimen
42492,310.1461,176.1314,0.8230987,43904,0.6658936,823.796,Kecimen
42256,323.1896,172.5759,0.8454988,44743,0.6980309,849.728,Kecimen
64380,366.9648,227.7716,0.7840556,66125,0.6643757,981.544,Kecimen


In [18]:
raisin_recipe <- recipe(Class ~ . , data = raisin_subset) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

In [19]:
# creating an empty tibble which will store the accuracy results
accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

In [20]:
# create a model 
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) %>%
     set_engine("kknn") %>%
     set_mode("classification")


In [21]:
# 5-fold cross-validation
cancer_vfold <- vfold_cv(raisin_subset, v = 5, strata = Class)

# total # of predictors
n_total <- length(names)

# creating to store selected predictors
selected <- c()

In [22]:
#loops
# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()

ERROR: Error in parse(text = x, srcfile = src): <text>:7:0: unexpected end of input
5:     accs <- list()
6:     models <- list()
  ^


3. Describing at least one way to visualize the results

visualization

In [ ]:
We can vizualize the accuracy based on the number of predicators using the knn alogirthim 